In [9]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 393.8 kB/s eta 0:00:33
     --------------------------------------- 0.0/12.8 MB 393.8 kB/s eta 0:00:33
     --------------------------------------- 0.1/12.8 MB 409.6 kB/s eta 0:00:32
     --------------------------------------- 0.1/12.8 MB 403.5 kB/s eta 0:00:32
     --------------------------------------- 0.1/12.8 MB 403.5 kB/s eta 0:00:32
     --------------------------------------- 0.1/12.8 MB 425.1 kB/s eta 0:00:30
     --------------------------------------- 0.2/12.8 MB 447.8 kB/s eta 0:00:29
      -------------------------------------- 0.2/12.8 MB 472.6 kB/s eta 0:00:27
      -------------------------------------- 0.2/12.8 MB 472.6 kB/s eta 0:00:27
      -------------------------------------- 0.2/12.8 MB 415.7 kB/s eta 0:00:31
      -------------------------------------- 0.2/12.8 MB

In [2]:
import spacy

# Load the English NLP model
nlp = spacy.load('en_core_web_sm')

# Process a piece of text
text = "benston is going to church."
doc = nlp(text)

# Extract entities from the text
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


benston 0 7 PERSON


In [6]:
import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example
import random

# 1. Preparing the Training Data:
TRAIN_DATA = [
    ("Georgia said she's never been to Georgia.", {"entities": [(0, 7, "PERSON"), (38, 45, "GPE")]}),
    ("Jordan will travel to Jordan next month.", {"entities": [(0, 6, "PERSON"), (24, 30, "GPE")]}),
    ("I met Brooklyn in a cafe in Brooklyn.", {"entities": [(6, 14, "PERSON"), (30, 38, "GPE")]}),
    ("Chelsea, can you guide me to Chelsea?", {"entities": [(0, 7, "PERSON"), (31, 38, "GPE")]}),
    ("Austin's favorite city is not Austin.", {"entities": [(0, 6, "PERSON"), (31, 37, "GPE")]}),
    ("Did Virginia call the governor of Virginia?", {"entities": [(4, 12, "PERSON"), (35, 43, "GPE")]}),
    ("I saw Florence while traveling to Florence.", {"entities": [(6, 14, "PERSON"), (34, 42, "GPE")]}),
    ("Victoria will meet you at the Victoria station.", {"entities": [(0, 8, "PERSON"), (31, 39, "GPE")]}),
    ("Madison wrote a letter to the mayor of Madison.", {"entities": [(0, 7, "PERSON"), (41, 48, "GPE")]}),
    ("Phoenix sent a postcard from Phoenix.", {"entities": [(0, 7, "PERSON"), (31, 38, "GPE")]}),
    ("Sydney from Sydney sent you a gift.", {"entities": [(0, 6, "PERSON"), (12, 18, "GPE")]}),
    ("Orlando told Orlando's story in Orlando.", {"entities": [(0, 7, "PERSON"), (14, 21, "PERSON"), (35, 42, "GPE")]}),
    ("Is Regina the queen of Regina?", {"entities": [(3, 9, "PERSON"), (24, 30, "GPE")]}),
    ("Dallas is planning a trip to Dallas.", {"entities": [(0, 6, "PERSON"), (31, 37, "GPE")]}),
    ("I found Asia's diary while in Asia.", {"entities": [(8, 12, "PERSON"), (30, 34, "GPE")]}),
    ("Houston, we have a problem in Houston.", {"entities": [(0, 7, "PERSON"), (31, 38, "GPE")]}),
    ("Is Denver from Denver?", {"entities": [(3, 9, "PERSON"), (15, 21, "GPE")]}),
    ("London wants to live outside of London.", {"entities": [(0, 6, "PERSON"), (32, 38, "GPE")]}),
    ("I heard a song about Adelaide in Adelaide.", {"entities": [(20, 28, "PERSON"), (32, 40, "GPE")]}),
    ("Can you find Alexandria in Alexandria?", {"entities": [(13, 24, "PERSON"), (28, 39, "GPE")]}),
    ("I met Dakota and we traveled to Dakota.", {"entities": [(6, 12, "PERSON"), (35, 41, "GPE")]}),
    ("Jersey said he lost his jersey in Jersey.", {"entities": [(0, 6, "PERSON"), (36, 42, "GPE")]}),
    ("Did you see Paris in Paris?", {"entities": [(12, 17, "PERSON"), (21, 26, "GPE")]}),
    ("I'll meet Dallas in the Dallas museum.", {"entities": [(10, 16, "PERSON"), (24, 30, "GPE")]}),
    ("Canberra told me Canberra's history.", {"entities": [(0, 8, "PERSON"), (13, 21, "PERSON")]}),
    ("I found a picture of Berlin in Berlin's gallery.", {"entities": [(21, 27, "PERSON"), (31, 37, "GPE")]}),
    ("Is there a difference between the two Oslos?", {"entities": [(37, 42, "GPE")]}),
    ("I met a guy named York in New York.", {"entities": [(18, 22, "PERSON"), (26, 34, "GPE")]}),
    ("Did you know that Rome wasn't built in a day?", {"entities": [(18, 22, "GPE")]}),
    ("I met a girl named India in India.", {"entities": [(18, 23, "PERSON"), (27, 32, "GPE")]}),
    ("Washington wrote a letter to Washington.", {"entities": [(0, 10, "PERSON"), (29, 39, "GPE")]}),
    ("I found a cat named Salem in Salem.", {"entities": [(20, 25, "PERSON"), (29, 34, "GPE")]}),
    ("Did you meet the new student, Kenya?", {"entities": [(30, 35, "PERSON")]}),
    ("I heard Cairo's music in a cafe in Cairo.", {"entities": [(7, 12, "PERSON"), (33, 38, "GPE")]}),
    ("Monaco said that Monaco is beautiful.", {"entities": [(0, 6, "PERSON"), (16, 22, "GPE")]}),
    ("I met a singer named Vienna in Vienna.", {"entities": [(21, 27, "PERSON"), (31, 37, "GPE")]}),
    ("Did you see the movie titled Rome in Rome?", {"entities": [(28, 32, "PERSON"), (36, 40, "GPE")]}),
    ("I met a dancer named Geneva in Geneva.", {"entities": [(20, 26, "PERSON"), (30, 36, "GPE")]}),
    ("Did you know that Oslo has a twin named Oslo?", {"entities": [(18, 22, "GPE"), (41, 45, "PERSON")]}),
    ("I met a chef named Tokyo in Tokyo.", {"entities": [(18, 23, "PERSON"), (27, 32, "GPE")]}),
    ("Barcelona is a friend from Barcelona.", {"entities": [(0, 9, "PERSON"), (26, 35, "GPE")]}),
    ("I met a writer named Denver in Denver.", {"entities": [(21, 27, "PERSON"), (31, 37, "GPE")]}),
    ("Did you meet the artist named Milan in Milan?", {"entities": [(29, 34, "PERSON"), (38, 43, "GPE")]}),
    ("I met a musician named Lisbon in Lisbon.", {"entities": [(23, 29, "PERSON"), (33, 39, "GPE")]}),
    ("Havana said that Havana is vibrant.", {"entities": [(0, 6, "PERSON"), (16, 22, "GPE")]}),
    ("I met a director named Madrid in Madrid.", {"entities": [(23, 29, "PERSON"), (33, 39, "GPE")]}),
    ("Did you see the play titled Berlin in Berlin?", {"entities": [(28, 34, "PERSON"), (38, 44, "GPE")]}),
    ("I met a producer named Athens in Athens.", {"entities": [(24, 30, "PERSON"), (34, 40, "GPE")]}),
    ("Moscow said that Moscow has a rich history.", {"entities": [(0, 6, "PERSON"), (16, 22, "GPE")]}),
    ("I met an actor named Dublin in Dublin.", {"entities": [(22, 28, "PERSON"), (32, 38, "GPE")]}),
    ("Did you meet the singer named Rome in Rome?", {"entities": [(29, 33, "PERSON"), (37, 41, "GPE")]}),
    ("I met a painter named Warsaw in Warsaw.", {"entities": [(23, 29, "PERSON"), (33, 39, "GPE")]}),
    ("Brussels said that Brussels is unique.", {"entities": [(0, 8, "PERSON"), (18, 26, "GPE")]}),
    ("I met a designer named Zurich in Zurich.", {"entities": [(25, 31, "PERSON"), (35, 41, "GPE")]}),
    ("Did you see the show titled Paris in Paris?", {"entities": [(28, 33, "PERSON"), (37, 42, "GPE")]}),
    ("I met a developer named Munich in Munich.", {"entities": [(26, 32, "PERSON"), (36, 42, "GPE")]}),
    ("Stockholm said that Stockholm is peaceful.", {"entities": [(0, 9, "PERSON"), (19, 28, "GPE")]}),
    ("I met a programmer named Oslo in Oslo.", {"entities": [(27, 31, "PERSON"), (35, 39, "GPE")]}),
    ("Did you meet the engineer named Vienna in Vienna?", {"entities": [(32, 38, "PERSON"), (42, 48, "GPE")]}),
    ("I met a scientist named Prague in Prague.", {"entities": [(25, 31, "PERSON"), (35, 41, "GPE")]}),
    ("Budapest said that Budapest has a river.", {"entities": [(0, 8, "PERSON"), (18, 26, "GPE")]}),
    ("I met a researcher named Lisbon in Lisbon.", {"entities": [(28, 34, "PERSON"), (38, 44, "GPE")]}),
    ("Did you meet the professor named Madrid in Madrid?", {"entities": [(33, 39, "PERSON"), (43, 49, "GPE")]}),
    ("I met a student named Athens in Athens.", {"entities": [(22, 28, "PERSON"), (32, 38, "GPE")]}),
    ("Copenhagen said that Copenhagen is cold.", {"entities": [(0, 10, "PERSON"), (20, 30, "GPE")]}),
    ("I met a teacher named Dublin in Dublin.", {"entities": [(23, 29, "PERSON"), (33, 39, "GPE")]}),
    ("Did you meet the principal named Rome in Rome?", {"entities": [(32, 36, "PERSON"), (40, 44, "GPE")]}),
    ("I met a principal named Warsaw in Warsaw.", {"entities": [(24, 30, "PERSON"), (34, 40, "GPE")]}),
    ("Brussels is a friend named Brussels from Brussels.", {"entities": [(0, 8, "PERSON"), (23, 31, "PERSON"), (37, 45, "GPE")]}),
    ("I met a friend named Zurich in Zurich.", {"entities": [(20, 26, "PERSON"), (30, 36, "GPE")]}),
    ("Did you see the movie star Paris in Paris?", {"entities": [(26, 31, "PERSON"), (35, 40, "GPE")]}),
    ("I met a celebrity named Munich in Munich.", {"entities": [(25, 31, "PERSON"), (35, 41, "GPE")]}),
    ("Stockholm is a celebrity from Stockholm.", {"entities": [(0, 9, "PERSON"), (29, 38, "GPE")]}),
    ("I met a star named Oslo in Oslo.", {"entities": [(17, 21, "PERSON"), (25, 29, "GPE")]})]


# 2. Setting up the Model:
# Load an existing model or create a blank one
# nlp = spacy.blank("en")
nlp = spacy.load("en_core_web_sm")

# Get the NER component
ner = nlp.get_pipe("ner")

# 3. Training the Model:

# Add labels to the NER component
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Train the model
optimizer = nlp.create_optimizer()
for itn in range(100):  # Number of iterations can be adjusted
    random.shuffle(TRAIN_DATA)
    losses = {}
    
    # Convert training data to 'Example' format
    examples = [Example.from_dict(nlp(text), annotations) for text, annotations in TRAIN_DATA]
    
    for batch in minibatch(examples, size=compounding(4.0, 32.0, 1.001)):
        nlp.update(batch, drop=0.5, sgd=optimizer, losses=losses)
    
    print(f"Iteration {itn + 1}, Losses {losses}")

# 4. Evaluating and Saving the Model:
model_path = "new_ner"
nlp.to_disk(model_path)

# 5. Using the Model with Potential Rule-based Post-processing:
nlp2 = spacy.load(model_path)
text = "Georgia went to Paris."
doc = nlp2(text)


c:\Users\jubal\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Chelsea, can you guide me to Chelsea?" with entities "[(0, 7, 'PERSON'), (31, 38, 'GPE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\jubal\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Did you see the movie titled Rome in Rome?" with entities "[(28, 32, 'PERSON'), (36, 40, 'GPE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\jubal\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning:

Iteration 1, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 63.96433381100953}
Iteration 2, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 24.896203418433455}
Iteration 3, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 20.5347392180814}
Iteration 4, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 14.272815660324898}
Iteration 5, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 10.172771664029678}
Iteration 6, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 2.275562183490154}
Iteration 7, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 5.892823024730345}
Iteration 8, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.3506649784160842}
Iteration 9, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 3.187074731004949}
Iteration 10, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 2.0303564909771294}
Iteration 11, Losses {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.

In [ ]:
model_path = "new_ner"
nlp2 = spacy.load(model_path)
doc = nlp2("benston is going to jordan")
for ent in doc.ents:
    print(ent.text, ent.label_)